<a href="https://colab.research.google.com/github/Leon-Av/yt-dlp_gradio-colab/blob/main/yt_dlp_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка необходимых пакетов
!pip install yt-dlp gradio
!sudo apt update
!sudo apt install -y ffmpeg

import gradio as gr
import os
from yt_dlp import YoutubeDL

# Создаем папку для загрузок
download_folder = "/content/downloads"
os.makedirs(download_folder, exist_ok=True)

def download_video(url):
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best',
        'outtmpl': f'{download_folder}/%(title)s.%(ext)s',
        'merge_output_format': 'mp4',
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=True)
            filename = ydl.prepare_filename(info)
            return filename
    except Exception as e:
        return str(e)

# Функция для очистки старых файлов
def cleanup():
    for file in os.listdir(download_folder):
        file_path = os.path.join(download_folder, file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
        except Exception as e:
            print(f'Ошибка при удалении {file_path}: {e}')

# Создаем интерфейс Gradio
with gr.Blocks() as demo:
    gr.Markdown("## YouTube Video Downloader")
    with gr.Row():
        url_input = gr.Textbox(label="URL")
    with gr.Row():
        download_button = gr.Button("Скачать/Download")
    with gr.Row():
        file_output = gr.File(label="Downloaded File")

    download_button.click(
        fn=download_video,
        inputs=url_input,
        outputs=file_output
    )

    demo.load(fn=cleanup, inputs=None, outputs=None)

# Запускаем приложение с публичной ссылкой
demo.launch(share=True)